In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from timeit import default_timer as timer

def diff_h(N,j,x,xjs):
    dx = x-xjs[j]
    temp1 = np.cos(N*dx/2)*np.cos(dx/2)*N*np.sin(dx/2)-np.sin(N*dx/2)
    return temp1/(2*N*np.sin(dx/2)**2)


def Dmatrix(N,xjs):
    D = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
                    if i == j:
                        D[i][j] = 0
                    else: 
                        D[i][j] = diff_h(N,j,xjs[i],xjs)
    for i in range(N):
         s = np.sum(D[i,:])
         D[i][i] = -s
    return D

def vfun(x):
    return np.exp(np.sin(np.pi*x))
vfun = np.vectorize(vfun)

def v_diff_true(x):
    return np.pi*np.cos(np.pi*x)*np.exp(np.sin(np.pi*x))

def fft_derivative(N, fun, L = 2*np.pi):
    
    h = L/N
    X = np.arange(N)*h
    f = fun(X)

    k = sp.fft.fftfreq(N,1/N) * 1j

    return np.real(sp.fft.ifft( k * sp.fft.fft(f) ))*2*np.pi/L



Ns = [100,200,300,400,500,800,1000]#,750,1000]
time_matrix = np.zeros(len(Ns),dtype=np.float64)
time_fft = np.zeros(len(Ns),dtype=np.float64)
reps = 20
for i,N in enumerate(Ns):
    xjs = np.array([j*2/N for j in range(N)]) 
    vx = vfun(xjs)
    Dmat = Dmatrix(N,xjs*np.pi)
    for _ in range(reps):
        t1 = timer()
        vx_diff_matrix = (Dmat @ vx)*np.pi
        t2 = timer()
        time_matrix[i] += (t2-t1)
    for _ in range(reps):
        h = 2/N
        X = np.arange(N)*h
        f = vfun(X)
        k = sp.fft.fftfreq(N,1/N) * 1j
        t1 = timer()
        vx_diff_fft = np.real(sp.fft.ifft( k * sp.fft.fft(f) ))*2*np.pi/2
        t2 = timer()
        time_fft[i] += (t2-t1)
plt.figure()
plt.loglog(Ns,time_fft/reps,label="FFT method")
plt.loglog(Ns,time_matrix/reps,label="Differentiation Matrix")
plt.legend()
plt.xlabel("N")
plt.ylabel("Time [s]")
plt.show()